In [ ]:
#hide
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

# Testing pytorch models

Faster R-CNN ResNet-50 FPN
Mask R-CNN ResNet-50 FPN

## IoU on Testset

In [ ]:
import torch
import torchvision.models.detection as models
import pytorch_lightning as pl

In [ ]:
import numpy as np
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from torchvision.transforms import ToTensor, Compose, ToPILImage, Normalize
from detection_nbdev.dataset import XMLDetectionDataModule, XMLDetectionDataset
from detection_nbdev.model import TorchVisionDetector

# Setup models
model_names = ['fasterrcnn_resnet50_fpn', 'retinanet_resnet50_fpn', 'maskrcnn_resnet50_fpn']

# Prepare data
data_dir = '../detection/data/'
image_transform = Compose([
    ToTensor(),
    lambda x: x.unsqueeze(0)])

dm = XMLDetectionDataModule(
    data_dir, 
    image_transform=ToTensor())
dm.setup(mode='use_dir')

# for mn in model_names:
#     model=TorchVisionDetector(mn, pretrained=True)
#     trainer = pl.Trainer(gpus='1', checkpoint_callback=False)
#     trainer.test(model, dm.test_dataloader())

## Visualizing Trajectory 

In [ ]:
import numpy as np
import cv2
import glob
import torchvision
from PIL import Image
from detection_nbdev.utils import visualize_bbox

model=TorchVisionDetector('maskrcnn_resnet50_fpn', pretrained=True)
model.eval();

In [ ]:
# Prepare data
root = '../detection/data/videoframes/*.jpg'
transform = torchvision.transforms.ToTensor()
files = glob.glob(root)

import os
# Prepare output
os.makedirs('./out', exist_ok=True)

In [ ]:
for i, file in enumerate(files):
    im = Image.open(file).convert("RGB")
    x = transform(im).unsqueeze(0)
    y_pred = model(x)
    
    bboxes = [list(map(int, _y)) for _y in y_pred[0]]
    if len(bboxes) > 0:
        im = Image.fromarray(visualize_bbox(
            np.asarray(im),
            bboxes[0],
            'ball'
        ))
    im.save(f'out/{i:06d}.png')

KeyboardInterrupt: 